In [8]:
#HAVE TO MAKE LESS SEARCHES!!!

#packages
!pip install -q serpapi
!pip install -q google-search-results

#imports
from serpapi import GoogleSearch
import pandas as pd
import pprint
import json

#set serpAPI key
apikey = "3a189e6568396017b356bee1e56fd5e0cc4b50d99a410d2245bad93bf8f40e6c"

#ITERATE THROUGH FACILITIES
#read place names and locations
place_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/healthcare_facilities_2023.csv')

#remove places without coordinates(requirement for review extraction)
place_data.dropna(subset=['coordinates'], inplace=True)

#retrieve place name
place_data['OU5name'] = place_data['OU5name'].apply(lambda x: x[3:])
#place_data = place_data.iloc[:5] #remove
place_data = place_data[place_data['OU5short']=='Mediclinic Durbanville Hosp']
#declare for output
All_data_output = []

#LOOP THROUGH ALL FACILITIES
for facility in place_data.iterrows():
  #set variables
  place_name = str(facility[1]['OU5name'])
  place_latitude = str(facility[1]['latitude'])
  place_longitude = str(facility[1]['longitude'])
  zoom = "17z"
  location = "@" + place_latitude + "," + place_longitude + "," + zoom
  #FIND PLACE_IDs FOR FIRST SEARCH RESULT
  #set search parameters
  params = {
    "engine": "google_maps",
    "q": place_name,
    "ll": location,
    "type": "search",
    "api_key": apikey
  }

  #perform search
  search = GoogleSearch(params)
  results = search.get_dict()
  #pprint.pprint(results)
  #Find place_ID (diff between place and local results)
  if 'place_results' in results:
    place_ID = results["place_results"]["place_id"]

  else:
    place_ID = results["local_results"][0]["place_id"]

  #FIND REVIEWS OF FIRST PAGE

  #set parameters
  params = {
    "engine": "google_maps_reviews",
    "place_id": place_ID,
    "api_key": apikey
  }

  #perform search
  search = GoogleSearch(params)
  results = search.get_dict()

  #retrieve place information (used in export)
  place_info = results["place_info"]
  #retrieve topics
  if 'topics' in results:
    topics = results["topics"]

  #retrieve reviews page 1
  reviews_filtered = []

  #filter only required review data
  if 'reviews' in results:
    reviews1 = results["reviews"]
    for review in reviews1:
        if 'extracted_snippet' in review:
          paragraph = review['extracted_snippet']['original']
        else:
          paragraph = "No Review Text"
        if 'iso_date' in review:
          date = review['iso_date']
        else:
          date = "No Date"
        if 'rating' in review:
          rating = review['rating']
        else:
          rating = "No Rating"

        review_info = {'rating': rating, 'date': date, 'paragraph': paragraph}
        reviews_filtered.append(review_info)
  else:
    reviews_filtered = []


  #set next page token
  if 'serpapi_pagination' in results:
    next_page_token = results["serpapi_pagination"]["next_page_token"]
  else:
    next_page_token = 0

  #GET REVIEWS FROM PAGE 2 ONWARD (MAX 100)

  if next_page_token == 0 or not reviews_filtered:
    print("no more reviews")
  else:
    for i in range(9):
      if next_page_token == 0:
        print("no more reviews")
        break
      else:
        #set parameters
        params = {
          "engine": "google_maps_reviews",
          "place_id": place_ID,
          "next_page_token": next_page_token,
          "api_key": apikey
        }

        #perform search
        search = GoogleSearch(params)
        results = search.get_dict()

        #filter only required review data
        if 'reviews' in results:
          reviews1 = results["reviews"]
          for review in reviews1:
              if 'extracted_snippet' in review:
                paragraph = review['extracted_snippet']['original']
              else:
                paragraph = "No Review Text"
              if 'iso_date' in review:
                date = review['iso_date']
              else:
                date = "No Date"
              if 'rating' in review:
                rating = review['rating']
              else:
                rating = "No Rating"

              review_info = {'rating': rating, 'date': date, 'paragraph': paragraph}
              reviews_filtered.append(review_info)

        #set next page token
        if 'serpapi_pagination' in results:
          next_page_token = results["serpapi_pagination"]["next_page_token"]
        else:
          next_page_token = 0

  #pprint.pprint(reviews_filtered) #remove

  #CREATE NEAT OUTPUT FOR EACH FACILITY
  current_facility_output = {
      'name': place_info['title'],
    'place_info': place_info,
      'topics': topics,
    'reviews': reviews_filtered
  }
  All_data_output.append(current_facility_output)
#pprint.pprint(All_data_output) #remove

#WRITE TO JSON FILE
# Specify the filename for the JSON file
filename = 'reviews.json'

# Write combined_data to the JSON file (use 'a' instead of 'w' for appending rather than overwriting)
with open(filename, 'w') as json_file:
    json.dump(All_data_output, json_file, indent=4)

[{'date': '2024-03-19T09:04:06Z',
  'paragraph': 'Came in with my son’s finger practically off trying to get '
               'someone to assist. Took a long while for a nurse to attend. '
               'With an emergency room that was empty. Then doctor was waiting '
               'for my son in surgery for 30 minutes while they took him to '
               'the ward. Next day waiting for medication for over an hour '
               'when the doctor discharged my son at 9am and 11:00 still '
               'waiting on medication. Really a very slack working ethic '
               'manner at the hospital is the overall experience. Going back '
               'for a follow up at Mediclinic the past Saturday, I had edited '
               'the review and put an additional star to the review. It was an '
               'improvement from the Monday. The ER doctor and  nurses in the '
               'ER were helpful and provided great care the morning. Thank '
               'you.',
  'ra